In [1]:
import cv2
import numpy as np
import operator
import os

In [2]:
#module level variables, they have module level scope.
MIN_CONTOUR_AREA = 100
RESIZED_IMAGE_WIDTH = 20
RESIZED_IMAGE_HEIGHT = 30

Contours : can be explained simply as a curve joining all the continuous points (along the boundary), having same color or intensity. The contours are a useful tool for shape analysis and object detection.

OpenCV, finding contours is like finding white object from black background.

In [3]:
class ContourWithData():

    # member variables
    npaContour = None           # contour
    boundingRect = None         # bounding rect for contour
    intRectX = 0                # bounding rect top left corner x location
    intRectY = 0                # bounding rect top left corner y location
    intRectWidth = 0            # bounding rect width
    intRectHeight = 0           # bounding rect height
    fltArea = 0.0               # area of contour

    def calculateRectTopLeftPointAndWidthAndHeight(self):               # calculate bounding rect info
        [intX, intY, intWidth, intHeight] = self.boundingRect
        self.intRectX = intX
        self.intRectY = intY
        self.intRectWidth = intWidth
        self.intRectHeight = intHeight

    def checkIfContourIsValid(self):                            # this is oversimplified, for a production grade program
        if self.fltArea < MIN_CONTOUR_AREA: return False        # much better validity checking would be necessary
        return True

In [4]:
def main():
    allcontours=[]    #declaring empty lists.
    validcontours=[]
    try:
        classifications=np.loadtxt("D:/ML_datasets/character_recog(internship_p-1)/classifications.txt",np.float32) #load the txt file ,as float 32 bit.
    except:
        print ("unable to open classifications.txt\n")
        os.system("pause") # exiting program
        return
    
    try:
        flattenedimages=np.loadtxt("D:/ML_datasets/character_recog(internship_p-1)/flattened_images.txt",np.float32)
    except:
        print ("unable to open flattened images txt file\n")
        os.system(pause)
        return
    
    classifications=classifications.reshape(classifications.size,1) #reshape numpy array to 1D
    
    knear=cv2.ml.KNearest_create() #create knn object using open cv.
    knear.train(flattenedimages,cv2.ml.ROW_SAMPLE,classifications)
    test_img = cv2.imread("D:/ML_datasets/character_recog(internship_p-1)/test_all.png") #read testing numbers image
    
    if test_img is None:                           # if image was not read successfully
        print ("error: image not read from file \n\n")        # print error message to std out
        os.system("pause")                                  # pause so user can see error message
        return                                              # and exit function (which exits program)
    
    gray_img = cv2.cvtColor(test_img,cv2.COLOR_BGR2GRAY)
    blur_img=cv2.GaussianBlur(gray_img,(5,5),0) #blur
    
    
    # filter image from grayscale to black and white
    imgthresh = cv2.adaptiveThreshold(blur_img,                           # input image
                                      255,                                  # make pixels that pass the threshold full white
                                      cv2.ADAPTIVE_THRESH_GAUSSIAN_C,       # use gaussian rather than mean, seems to give better results
                                      cv2.THRESH_BINARY_INV,                # invert so foreground will be white, background will be black
                                      11,                                   # size of a pixel neighborhood used to calculate threshold value
                                      2)                                    # constant subtracted from the mean or weighted mean
    
    copythresh = imgthresh.copy()  # make a copy as findcontours will modify it.
    
    npaContours, npaHierarchy = cv2.findContours(copythresh,   # input image, make sure to use a copy since the function will modify this image in the course of finding contours
                                                 cv2.RETR_EXTERNAL,         # retrieve the outermost contours only
                                                 cv2.CHAIN_APPROX_SIMPLE)   # compress horizontal, vertical, and diagonal segments and leave only their end points
    
    
    for x in npaContours:
        contour = ContourWithData()                                             # instantiate a contour with data object
        contour.npaContour = x                                           # assign contour to contour with data
        contour.boundingRect = cv2.boundingRect(contour.npaContour)     # get the bounding rect
        contour.calculateRectTopLeftPointAndWidthAndHeight()                    # get bounding rect info
        contour.fltArea = cv2.contourArea(contour.npaContour)           # calculate the contour area
        allcontours.append(contour)                                     # add contour with data object to list of all contours with data
    
    for y in allcontours:              #for all contours
        if y.checkIfContourIsValid():   #check if valid
            validcontours.append(y)      #append to valid contour list
            
    validcontours.sort(key=operator.attrgetter("intRectX")) #sort contours from left to right
    
    strFinalString = ""         # declare final string, this will have the final number sequence by the end of the program

    for contour in validcontours:            # for each contour
                                                # draw a green rect around the current char
        cv2.rectangle(test_img,                                        # draw rectangle on original testing image
                      (contour.intRectX, contour.intRectY),     # upper left corner
                      (contour.intRectX + contour.intRectWidth, contour.intRectY + contour.intRectHeight),      # lower right corner
                      (0, 255, 0),              # green
                      2)                        # thickness
        
        imgROI = imgthresh[contour.intRectY : contour.intRectY + contour.intRectHeight,     # crop char out of threshold image
                           contour.intRectX : contour.intRectX + contour.intRectWidth]

        imgROIResized = cv2.resize(imgROI, (RESIZED_IMAGE_WIDTH, RESIZED_IMAGE_HEIGHT))             # resize image, this will be more consistent for recognition and storage

        npaROIResized = imgROIResized.reshape((1, RESIZED_IMAGE_WIDTH * RESIZED_IMAGE_HEIGHT))      # flatten image into 1d numpy array

        npaROIResized = np.float32(npaROIResized)       # convert from 1d numpy array of ints to 1d numpy array of floats

        retval, npaResults, neigh_resp, dists = knear.findNearest(npaROIResized, k = 1)     # call KNN function find_nearest

        strCurrentChar = str(chr(int(npaResults[0][0])))                                             # get character from results

        strFinalString = strFinalString +" "+strCurrentChar            # append current char to full string
        
        
    print ("\n"+strFinalString+"\n")    #show full string
    cv2.imshow("image testing numbers ",test_img)   #show input image with green boxes around found digits.
    cv2.waitKey(0)
    cv2.destroyAllWindows()
        
    return

if __name__=="__main__":
    main()


 A 0 A 0 A 0 A 0 A 0 1 B 1 B 1 B 1 B 1 B 2 C 2 C 3 C 2 2 C 2 C D 3 4 D 3 D D 3 4 E 3 D 5 E 5 4 F 6 E 4 E E 4 F 6 7 G 5 F G 5 F 5 7 F 8 H G 6 9 8 H G 6 G I 6 9 J 7 H I H 7 J H 7 K 8 I K I 8 L 8 I 9 J L M 9 J J 9 M K N K K N L O O L L P M P Q M M N Q R N N O S R O P O T S U P Q T P V U Q R Q W V S R R X W T S S Y X U T Z T Y V U Z U W V V X W W Y X X Z Y Y Z Z

